In [1]:
import sqlalchemy
import pandas as pd
BL_engine=sqlalchemy.create_engine(
            "mysql+pymysql://%s:%s@localhost/%s" % ('jian', 'JubaPlus-2017', 'BigLots'))

In [26]:
a=pd.read_sql("show tables;",con=BL_engine).iloc[:,0].values.tolist()
a

['BL_CCPA_email',
 'BL_POS_Item',
 'BL_POS_Subclass',
 'BL_Rewards_Master',
 'NEall_id_from_crm_pos_until_20200620',
 'NEall_id_pos_from_20180204_to_20200509',
 'NEall_id_pos_from_20180204_to_20200516',
 'NEall_id_pos_until_20200627',
 'Pred_ExpV2_Activity_BL_id',
 'Pred_ExposureV2_GU_id',
 'Pred_ExposureV3_BL_id',
 'Pred_POS_Department',
 'all_NEall_id_pred_pos_2_1_pos_from_20180204_to_20200509',
 'all_NEall_id_pred_pos_2_1_pos_from_20180204_to_20200516',
 'all_NEall_id_pred_pos_2_1_pos_until_20200627',
 'all_NEall_id_pred_pos_2_2_pos_from_20180204_to_20200509',
 'all_NEall_id_pred_pos_2_2_pos_from_20180204_to_20200516',
 'all_NEall_id_pred_pos_2_2_pos_until_20200627',
 'crm_NEall_id_pos_from_20180204_to_20200509',
 'crm_NEall_id_pos_from_20180204_to_20200516',
 'crm_NEall_id_pos_until_20200627',
 'dept_id_list',
 'max_trans_NEall_id_pos_from_20180204_to_20200509',
 'max_trans_NEall_id_pos_from_20180204_to_20200516',
 'max_trans_NEall_id_pos_until_20200627',
 'pos_first_trans_20190503

In [4]:
list_tables=pd.read_sql("show tables;",con=BL_engine).iloc[:,0].values.tolist()
list_tables=[x for x in list_tables if x[-8:]=="20200627"]
list_tables

['NEall_id_pos_until_20200627',
 'all_NEall_id_pred_pos_2_1_pos_until_20200627',
 'all_NEall_id_pred_pos_2_2_pos_until_20200627',
 'crm_NEall_id_pos_until_20200627',
 'max_trans_NEall_id_pos_until_20200627',
 'pred_pos_dept_NEall_id_pos_until_20200627']

In [15]:
for t in list_tables:
    
    cols=pd.read_sql("desc %s"%t,con=BL_engine)['Field'].values.tolist()
    if "customer_id_hashed" in cols:
        count=pd.read_sql("select count(distinct customer_id_hashed) from %s;"%t,con=BL_engine).iloc[0,0]
    elif "id" in cols:
        count=pd.read_sql("select count(distinct id) from %s;"%t,con=BL_engine).iloc[0,0]
    else:
        print(t, "nothing")
        count=0
    print(t,count)
    

NEall_id_pos_until_20200627 493696
all_NEall_id_pred_pos_2_1_pos_until_20200627 493696
all_NEall_id_pred_pos_2_2_pos_until_20200627 490728
crm_NEall_id_pos_until_20200627 493696
max_trans_NEall_id_pos_until_20200627 493696
pred_pos_dept_NEall_id_pos_until_20200627 493696


In [16]:
df_ids_2_1=pd.read_sql("select distinct id as id_2_1 from all_NEall_id_pred_pos_2_1_pos_until_20200627;",con=BL_engine)
df_ids_2_2=pd.read_sql("select distinct id as id_2_2 from all_NEall_id_pred_pos_2_2_pos_until_20200627;",con=BL_engine)


In [19]:
df_ids_2_2.head(2)

,id_2_2
0,00001ed4962579d15869b5ec21ee8ecfa136d13b07c3f8...
1,00002443cd9a83fd2daf9ad3a2491e14627c1fb9bea392...


In [20]:
df_both=pd.merge(df_ids_2_1,df_ids_2_2,left_on="id_2_1",right_on="id_2_2",how="outer")
df_both.head(2)

,id_2_1,id_2_2
0,00001ed4962579d15869b5ec21ee8ecfa136d13b07c3f8...,00001ed4962579d15869b5ec21ee8ecfa136d13b07c3f8...
1,00002443cd9a83fd2daf9ad3a2491e14627c1fb9bea392...,00002443cd9a83fd2daf9ad3a2491e14627c1fb9bea392...


In [21]:
df_both[pd.isnull(df_both['id_2_1'])]

,id_2_1,id_2_2


In [23]:
list_missing_ids=df_both[pd.isnull(df_both['id_2_2'])]['id_2_1'].unique().tolist()
print(len(list_missing_ids))

2968


In [33]:
i_counter=0
df_missing_qc=pd.DataFrame()
for id_miss in list_missing_ids:
    id_sql="'"+id_miss+"'"
    df_missing_id_pos=pd.read_sql("select * from Pred_POS_Department where customer_id_hashed=%s"%id_sql,
                  con=BL_engine)
    df_missing_id_crm=pd.read_sql("select * from BL_Rewards_Master where customer_id_hashed=%s"%id_sql,\
                  con=BL_engine)
    
    df=pd.DataFrame({
        "min_transaction_dt":df_missing_id_pos['transaction_dt'].min(),
        "min_trans_order_since_18Q1":df_missing_id_pos['trans_order_since_18Q1'].min(),
        "sign_up_date":df_missing_id_crm['sign_up_date'].min()
    },index=[id_miss])
    df_missing_qc=df_missing_qc.append(df)
    i_counter+=1
    if i_counter%100==1:
        print(i_counter)
    
df_missing_qc=df_missing_qc.reset_index() 
df_missing_qc.shape

1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901


(2968, 4)

In [34]:
df_missing_qc.head(4)

,index,min_transaction_dt,min_trans_order_since_18Q1,sign_up_date
0,002776ad543b26a276c853906d3d1f6b7a055065bd5957...,2020-06-10,2,2020-06-10
1,00374ec0e0462b8e5454e6cbd8baf90ab155595313aa2d...,2020-06-13,2,2020-06-08
2,0044bc0890ce569d3b84cd4ae8878765423936233e2131...,2020-06-11,2,2020-06-11
3,004e0c46b931fe92ab117e0b74d2528b2cfe8989bc7d59...,2020-06-12,2,2020-06-12


In [35]:
for col in df_missing_qc.columns.tolist()[1:]:
    print(col,df_missing_qc[col].min())

min_transaction_dt 2020-06-07
min_trans_order_since_18Q1 2
sign_up_date 2009-09-12


In [36]:
a=df_missing_qc['min_transaction_dt'].unique().tolist()
a.sort()
a

[datetime.date(2020, 6, 7),
 datetime.date(2020, 6, 8),
 datetime.date(2020, 6, 9),
 datetime.date(2020, 6, 10),
 datetime.date(2020, 6, 11),
 datetime.date(2020, 6, 12),
 datetime.date(2020, 6, 13)]

In [38]:
df_unique_date=pd.read_sql("select distinct transaction_dt from Pred_POS_Department;",con=BL_engine)['transaction_dt'].tolist()
df_unique_date.sort()

In [39]:
df_unique_date

[datetime.date(2018, 2, 4),
 datetime.date(2018, 2, 5),
 datetime.date(2018, 2, 6),
 datetime.date(2018, 2, 7),
 datetime.date(2018, 2, 8),
 datetime.date(2018, 2, 9),
 datetime.date(2018, 2, 10),
 datetime.date(2018, 2, 11),
 datetime.date(2018, 2, 12),
 datetime.date(2018, 2, 13),
 datetime.date(2018, 2, 14),
 datetime.date(2018, 2, 15),
 datetime.date(2018, 2, 16),
 datetime.date(2018, 2, 17),
 datetime.date(2018, 2, 18),
 datetime.date(2018, 2, 19),
 datetime.date(2018, 2, 20),
 datetime.date(2018, 2, 21),
 datetime.date(2018, 2, 22),
 datetime.date(2018, 2, 23),
 datetime.date(2018, 2, 24),
 datetime.date(2018, 2, 25),
 datetime.date(2018, 2, 26),
 datetime.date(2018, 2, 27),
 datetime.date(2018, 2, 28),
 datetime.date(2018, 3, 1),
 datetime.date(2018, 3, 2),
 datetime.date(2018, 3, 3),
 datetime.date(2018, 3, 4),
 datetime.date(2018, 3, 5),
 datetime.date(2018, 3, 6),
 datetime.date(2018, 3, 7),
 datetime.date(2018, 3, 8),
 datetime.date(2018, 3, 9),
 datetime.date(2018, 3, 10),
